In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
# from main import load_training_data, fill_missing_values

# features, target, customerId = load_training_data()

# features = fill_missing_values(features)

# load data
df = pd.read_csv("data/labels_training.txt")
views = pd.read_csv("data/views.txt")
views = views.drop(columns=['imageZoom']) # discard imageZoom column since all 0 but 1 value
aggr_views = views.groupby(['customerId','productId']).sum() # aggregate the views of a customer of a product by summing

# add customer info
customers = pd.read_csv("data/customers.txt")
df = pd.merge(df, customers, left_on=['customerId'], right_on=['customerId'], how='left')
# add product info
products = pd.read_csv("data/products.txt")
products = products.drop(columns=['dateOnSite'])
df = pd.merge(df, products, left_on=['productId'], right_on=['productId'], how='left')
# add views info
df = pd.merge(df, aggr_views, right_index=True, left_on=['customerId', 'productId'])

print("Dropping all rows with nan")
df.dropna(inplace=True) # TODO: temporarily remove all rows with missing info
print(df.shape)

In [ ]:
df

In [ ]:
from argparse import Namespace
import pandas as pd
from sklearn.compose import make_column_transformer
from main import *

args = Namespace(data_frac=0.1, max_depth=3, model='BoostedTree', nestimators=100, nfolds=10, seed=7, verbose=0)

print(args)

np.random.seed(args.seed)

print("Loading data ...")
features, target, customerId = load_training_data(args)

## preprocessing features
# fill missing values
print("Filling missing values")
features = fill_missing_values(features)

# encode categorical variables
column_trans = make_column_transformer(
    (OneHotEncoder(), ["country", "brand", "productType", "isFemale", "isPremier", "onSale"]),
    remainder="passthrough",
)
print("Fitting column transformer on categorical variables")
column_trans.fit(features)

In [ ]:
print("Starting cross-validation")
kfold = GroupKFold(n_splits=args.nfolds)

aucs = []
for train_index, val_index in kfold.split(X=features, groups=customerId):
    break

In [ ]:
features

In [ ]:
train_features = column_trans.transform(features.iloc[train_index])
train_features

In [ ]:
train_targets = target.iloc[train_index]
train_targets

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=args.nestimators, random_state=args.seed, max_depth=args.max_depth, verbose=args.verbose)

In [ ]:
clf.fit(train_features, train_targets)

In [ ]:
val_features = column_trans.transform(features.iloc[val_index])
val_targets = target.iloc[val_index]

In [ ]:
val_pred = clf.predict_proba(val_features)[:,1]
auc = roc_auc_score(val_targets, val_pred)

In [ ]:
auc